# Script for training with decoupled shape parameters

In [ ]:
from decalib.deca import DECA
from decalib.datasets import datasets 
from decalib.utils import util
from decalib.utils.config import config_default as deca_cfg
from skimage.io import imread
from tqdm import tqdm
import torch
import numpy as np
import mediapipe as mp
import os
import cv2
import trimesh
import matplotlib.pyplot as plt

In [ ]:
def get_shape_params(image_path: str, device='cuda'):
    """Function for getting the shape paramters of an image
    """
    # Initialization
    deca = DECA()

    #Get the data in the right format
    test_img = datasets.TestData(image_path)
    deca_cfg.model.use_tex = False
    deca_cfg.model.extract_tex = False
    #Get the codedictionarys
    for i in tqdm(range(len(test_img))):
        name = test_img[i]['imagename']
        images_l = test_img[i]['image'].to(device)[None,...]
        with torch.no_grad(): 
            codedict = deca.encode(images_l)

    return codedict['shape']